## Imports

In [1]:
import pandas as pd
import plotly.graph_objects as go
from datetime import date

## Funções Auxiliares

In [2]:
def make_graph(df, x_prop, y_prop_confirmados, y_prop_media_movel):
    fig = go.Figure()

    fig.add_trace(go.Bar(
        x = df[x_prop],
        y = df[y_prop_confirmados],
        name = 'Casos Confirmados'
    ))

    fig.add_trace(go.Scatter(
        x = df[x_prop],
        y = df[y_prop_media_movel],
        fillcolor = '#ff0000',
        name = 'Média Móvel'
    ))

    fig.update_layout(
        margin=dict(t=40, r=20, b=40, l=20),
        legend=dict(
            orientation="h",
            xanchor="center",
            x=0.5
        )
    )
    
    return fig

## Brasil

In [3]:
url_br = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv'
df_br = pd.read_csv(url_br)\
    .rename({'Country/Region': 'Country'}, axis=1)\
    .query('Country == "Brazil"')\
    .iloc[:,4:].T.reset_index()

df_br.columns = ['Data', 'Acumulado']
df_br.head(3)

,Data,Acumulado
0,1/22/20,0
1,1/23/20,0
2,1/24/20,0


In [4]:
df_br['Quantidade'] = df_br['Acumulado'] - df_br['Acumulado'].shift(1).fillna(0)
df_br['MA'] = df_br['Quantidade'].rolling(window=7).mean()

In [5]:
def convert_date(str_date):
    month, day, year = str_date.split('/')
    return date(int('20' + year), int(month), int(day)).isoformat()

df_br['Data'] = df_br['Data'].apply(convert_date)

In [6]:
make_graph(df_br, 'Data', 'Quantidade', 'MA')\
    .write_image("Covid-19_BR_Confirmados.png", format="png", width=800, height=350, scale=2)

## Espírito Santo

In [7]:
# Carregar o DataFrame
url_es = 'https://bi.s3.es.gov.br/covid19/MICRODADOS.zip'
df_es = pd.read_csv(url_es, sep = ';', encoding = 'cp1252')\
    .query('Classificacao == "Confirmados"')

In [8]:
# Definição de data dos confirmados
df_es['DataConfirmado'] = df_es['DataColeta_RT_PCR']\
    .combine_first(df_es['DataColetaTesteRapido'])\
    .combine_first(df_es['DataColetaSorologia'])\
    .combine_first(df_es['DataColetaSorologiaIGG'])\
    .combine_first(df_es['DataDiagnostico'])

df_es.head(3)

,DataNotificacao,DataCadastro,DataDiagnostico,DataColeta_RT_PCR,DataColetaTesteRapido,DataColetaSorologia,DataColetaSorologiaIGG,DataEncerramento,DataObito,Classificacao,...,ViagemInternacional,ProfissionalSaude,PossuiDeficiencia,MoradorDeRua,ResultadoRT_PCR,ResultadoTesteRapido,ResultadoSorologia,ResultadoSorologia_IGG,TipoTesteRapido,DataConfirmado
1,2021-10-11,2021-10-11,2021-10-11,NaN,2021-10-11,NaN,NaN,NaN,NaN,Confirmados,...,Não Informado,Não,Não,Não,Não Informado,Positivo,Não Informado,Não Informado,Teste rápido Antígeno,2021-10-11
12,2021-10-11,2021-10-11,2021-10-04,NaN,2021-10-08,NaN,NaN,NaN,NaN,Confirmados,...,Não Informado,Não,Não,Não,Não Informado,Positivo,Não Informado,Não Informado,Teste rápido Antígeno,2021-10-08
17,2021-10-11,2021-10-11,2021-10-05,NaN,2021-10-11,NaN,NaN,NaN,NaN,Confirmados,...,Não,Não,Não,Não,Não Informado,Positivo,Não Informado,Não Informado,Teste rápido Antígeno,2021-10-11


In [9]:
df_es = df_es.groupby('DataConfirmado')\
    .size()\
    .reset_index()\
    .rename({'DataConfirmado': 'Data', 0: 'Quantidade'}, axis=1)

df_es.head(3)

,Data,Quantidade
0,1980-12-02,1
1,2000-04-01,1
2,2000-12-17,1


In [10]:
# Remove datas erradas
today = date.today().isoformat()
df_es = df_es.query('Data > "2020-01-01" & Data < @today ')
df_es.head(3)

,Data,Quantidade
22,2020-01-02,4
23,2020-01-04,2
24,2020-01-05,6


In [11]:
df_es['MA'] = df_es['Quantidade'].rolling(window=7).mean()

In [12]:
make_graph(df_es, 'Data', 'Quantidade', 'MA')\
    .write_image("Covid-19_BR_ES_Confirmados.png", format="png", width=800, height=350, scale=2)